# Evaluating Classic Controllers

Evaluate non-learning controllers on the [Pendulum](https://gymnasium.farama.org/environments/classic_control/pendulum/) swing-up task using Myriad's evaluation API.

The pendulum starts hanging randomly and the goal is to swing it upright. We compare three strategies:
- **Random**: applies random torque each step
- **Bang-Bang**: a damping controller that applies maximum torque opposing the angular velocity
- **PID**: a proportional controller that smoothly damps angular velocity

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Video, display

from myriad import create_eval_config, evaluate
from myriad.envs import get_env_info
from myriad.utils.rendering import render_episode_to_video

## Run Evaluations

Each agent is evaluated over 50 rollouts. Agent-specific kwargs like `obs_field` and `kp` are passed as flat keyword arguments — Myriad routes them to the agent config automatically.

In [ ]:
agents = {
    "Random": dict(agent="random"),
    "Bang-Bang": dict(agent="bangbang", obs_field="theta_dot", threshold=0.0, invert=True),
    "PID": dict(agent="pid", obs_field="theta_dot", setpoint=0.0, kp=1.0, dt=0.05),
}

results = {}
for label, kwargs in agents.items():
    config = create_eval_config(env="pendulum-control", eval_rollouts=50, seed=0, **kwargs)
    results[label] = evaluate(config)
    print(f"{label}: {results[label]}\n")

## Compare Results

Plot mean return with standard deviation error bars. Higher (less negative) is better — the pendulum reward penalises distance from upright and torque magnitude.

In [ ]:
names = list(results.keys())
means = [results[n].mean_return for n in names]
stds = [results[n].std_return for n in names]

plt.bar(names, means, yerr=stds, capsize=5, color=["#999", "#e74c3c", "#3498db"])
plt.ylabel("Mean Return")
plt.title("Pendulum: Classic Controllers")
plt.tight_layout()
plt.show()

## Render Videos

Re-run each agent for a single episode with `return_episodes=True`, then use `render_episode_to_video` to produce an MP4. The environment registry provides the appropriate rendering function via `get_env_info`.

In [ ]:
render_fn = get_env_info("pendulum-control").render_frame_fn

for label, kwargs in agents.items():
    config = create_eval_config(env="pendulum-control", eval_rollouts=1, seed=0, **kwargs)
    results = evaluate(config, return_episodes=True)
    episode = {k: v[0] for k, v in results.episodes.items()}

    path = render_episode_to_video(episode, render_fn, f"pendulum_{kwargs['agent']}.mp4", fps=20)
    print(label)
    display(Video(str(path), embed=True, width=300))